In [ ]:
# +------------------------------+ #
# 資料設定值
weight_vector_title = 3
weight_vector_content = 1
weight_vector_tag = 2
weight_vector_tf = 2
# 設定 Milvus collection 名稱和維度
collection_name = 'trello_finder_v2'
# 設定分區名稱
partition_name = 'import'
# +------------------------------+ #

In [ ]:
# 引入套件
import pandas as pd
import numpy as np

# 讀取 json 檔案包含轉換向量
df = pd.read_json("../backup/vector_all.json")

In [ ]:
# Convert 'vector_content', 'vector_title', 'vector_tag', 'vector_tf' to NumPy ndarray
def convert_to_np_array(lst):
    # 檢查 lst 是否為 None 或為具有值的列表
    if lst is None:
        return np.zeros(768)
    elif isinstance(lst, list):
        return np.array(lst)
    else:
        return np.zeros(768)


# 將 'vector_content' 轉換為 NumPy 的 ndarray，若為 None 則轉換為長度 768 的 0 陣列
df["np_vector_content"] = df["vector_content"].apply(convert_to_np_array)

# 將 'vector_title' 轉換為 NumPy 的 ndarray，若為 None 則轉換為長度 768 的 0 陣列
df["np_vector_title"] = df["vector_title"].apply(convert_to_np_array)

# 將 'vector_tag' 轉換為 NumPy 的 ndarray，若為 None 則轉換為長度 768 的 0 陣列
df["np_vector_tag"] = df["vector_tag"].apply(convert_to_np_array)

# 將 'vector_tf' 轉換為 NumPy 的 ndarray，若為 None 則轉換為長度 768 的 0 陣列
df["np_vector_tf"] = df["vector_tf"].apply(convert_to_np_array)

In [ ]:
df

In [ ]:
# Define a function to search the vector
def search_array(arr_title, arr_content, arr_tag, arr_tf):
    if arr_title.sum() == 0:
        weight_vector_title = 0
    if arr_content.sum() == 0:
        weight_vector_content = 0
    if arr_tag.sum() == 0:
        weight_vector_tag = 0
    if arr_tf.sum() == 0:
        weight_vector_tf = 0

    return (weight_vector_title * arr_title + weight_vector_content * arr_content + weight_vector_tag * arr_tag + weight_vector_tf * arr_tf) / (weight_vector_title + weight_vector_tag + weight_vector_content + weight_vector_tf)


# Create a new column "vector_search" with default value None
df['vector_search'] = None

# Define a function to apply the search_array function to each row


def apply_search_array(row):
    return search_array(row['np_vector_title'], row['np_vector_content'], row['np_vector_tag'], row['np_vector_tf'])


# Apply the function to each row and update the "vector_search" column
df['vector_search'] = df.apply(apply_search_array, axis=1)

In [ ]:
from pymilvus import DataType, FieldSchema, CollectionSchema, Collection, connections
# 設定 Milvus 的連線資訊
connections.connect(alias="default")
milvus_collection = Collection(collection_name)

In [ ]:
errorlist = []

In [ ]:
# 將資料存入 Milvus
for idx, row in df.iterrows():
    try:

        insert_milvus_title = row["title"]
        if (len(insert_milvus_title) > 500):
            insert_milvus_title = insert_milvus_title[:497] + "..."

        try:
            # 插入 Milvus
            status = milvus_collection.insert({
                "id": str(idx),
                "track_id": str(idx),
                "value": row["vector_search"],
                "title": insert_milvus_title,
            }, partition_name=partition_name)

            if (status.insert_count != 1):
                print("Milvus Insert Fail")
                print(status)
                errorlist.append(idx)
            # else:
                # print("Done \n--------")

        except Exception as exp:
            print("--------------------")
            print(str(idx), "|", len(str(idx)))
            print("Vector |", len(row["vector"]))
            print(insert_milvus_title, "|", len(insert_milvus_title))
            print("--------------------")
            print("Milvus Insert Fail")
            print(exp)
            errorlist.append(idx)

    except Exception as exp:
        print("Milvus Insert Fail")
        print(exp)
        errorlist.append(idx)

In [ ]:
errorlist

---


In [ ]:
for idx, row in df.iloc[errorlist].iterrows():
    print(idx)
    insert_milvus_title = row["title"]
    if (len(insert_milvus_title) > 75):
        insert_milvus_title = insert_milvus_title[:71] + "..."
    print(len(insert_milvus_title))
    print(row["title"])

In [ ]:
expr = 'id in ["21846","8591"]'
milvus_collection.delete(expr)